## (Test) Calculate a gradient of this energy
large gradient components should force element to be swapped along gradient

# Binary puzzle/ Binario/ Takuzu puzzle
## Bruteforce puzzle generator (xD)
https://en.wikipedia.org/wiki/Takuzu

Rules:
1. Each row or column should have equal number of 1s and 0s
1. Cannot have more than 2 grouped entries horizontally or vertically

Approach:
1) Is checked via sum on proper axis
2) Is checked via convolution/cross-correlation

NOTE: Cannot easily batch convolutions using scipy. <br> 
NOTE: For fast approach see same implementation for GPU using Pytorch: [Binario_puzzle_generator_GPU.ipynb](../../multiprocessing/GPU/Binario_puzzle_generator_GPU.ipynb) (Bit better quality. very fast)

In [1]:
from scipy import signal, correlate
import numpy as np

three_horiz = np.array([[0,0,0],
                        [1,1,1],
                        [0,0,0]])

three_vert  = np.array([[0,1,0],
                        [0,1,0],
                        [0,1,0]])

nb_cnt_3= lambda puzz, ker: (signal.convolve2d(puzz, ker, boundary='fill', mode='same', fillvalue=0) == 3 ).astype(int)

def puzzle_ok_CPU(puzz):
    D = puzz.shape[0]//2
    # check if any entries have 2 neighbors (+1 self)
    neighb_failed  = (  nb_cnt_3(  puzz, three_horiz) |
                        nb_cnt_3(  puzz, three_vert ) |
                        nb_cnt_3(1-puzz, three_horiz) | # invert 0s and 1s (for sum to work)
                        nb_cnt_3(1-puzz, three_vert ) ) # invert 0s and 1s
    
    three_failed = np.any(neighb_failed)
    # check element count. 
    # in case of even N num 1s = num 0s. 
    # we dont count 0s
    vert_num_elems_failed   = np.any(np.sum(puzz, axis = 0) != D)
    horiz_num_elems_failed  = np.any(np.sum(puzz, axis = 1) != D)
    num_failed = vert_num_elems_failed or horiz_num_elems_failed

    return ~(three_failed or num_failed)

puzz = np.array([[1,0,1,0,1,0],
                 [0,1,0,0,1,1],
                 [1,0,0,1,0,1],
                 [0,1,1,0,1,0],
                 [0,0,1,1,0,1],
                 [1,1,0,1,0,0]])

print(f'Puzzle solved properly: {puzzle_ok_CPU(puzz)}')


Puzzle solved properly: True


In [19]:
from tqdm import tqdm
N = 6
B = 10_000_000
puzzles = np.uint8(np.random.rand(B,N,N)>0.5)
for i in tqdm(range(B)):
    np.random.seed(i)
    puzz = puzzles[i]
    if puzzle_ok_CPU(puzz):
        print(puzz)
        print(i)
        break

 70%|███████   | 7041061/10000000 [10:12<04:17, 11490.10it/s]

[[0 1 1 0 1 0]
 [0 1 0 1 0 1]
 [1 0 1 1 0 0]
 [0 1 0 0 1 1]
 [1 0 0 1 1 0]
 [1 0 1 0 0 1]]
7041061


# Iterative approach (Ising like?)
Find:
* 1s that can be swapped and in which direction (4 fields)
* 0s that can be swapped and in which direction (4 fields)
* unhappy/agitated 1s (more than 1 neighbor)
* unhappy/agitated 1s (not optimal row/column count)

In [2]:
from misc_tools.print_latex import print_tex

def swap_2_elements(arr, pos1, pos2):
    pos1 ,pos2 = tuple(pos1), tuple(pos2)
    temp = arr[pos1]
    arr[pos1] = arr[pos2]
    arr[pos2] = temp

def random_pos(arr2D, num, debug = False):
    # ~flattens array. generates flatten indices, unflattens indices back to shape.
    # returns [\vec{r}_1, \vec{r}_2, ..] 
    ij_flat = np.random.choice(arr2D.size, num, replace=False)
    ret = np.array(np.unravel_index(ij_flat, arr2D.shape)).T
    if debug: print(ret)
    return ret

input example : 
>>> arr_T = np.array([[r'\vec{v}_1', r'\vec{v}_2']]).T
>>> print_tex(arr_T,'=', np.arange(1,5).reshape(2,-1)/4, r'; symbols: \otimes, \cdot,\times')
output: 


<IPython.core.display.Math object>

## Define kernels for a gradient

## Modify proper puzzle by swapping 2 elems

In [15]:
puzz_c = puzz.copy()
np.random.seed(1)
swap_2_elements(puzz_c,(3,2),(2,2))#*random_pos(puzz_c,2, True)
swap_2_elements(puzz_c,(4,4),(4,5))

h_constr = lambda arr: np.sum(arr, axis= 1, keepdims=True)
v_constr = lambda arr: np.sum(arr, axis= 0, keepdims=True)

print_tex(puzz ,r'\rightarrow', puzz_c, h_constr(puzz_c))
print_tex(np.sum(puzz, axis= 0, keepdims=True), r'\rightarrow', v_constr(puzz_c))
N = puzz.shape[-1]

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Check which 1s can be swapped

In [16]:
k_n_l = np.array([[0 , 0, -0],
                  [1 , 1,  0],
                  [0 , 0, -0]])
k_n_r = 1*np.fliplr(k_n_l)

k_n_u = np.array([[0 , 1, -0],
                  [0 , 1,  0],
                  [0 , 0, -0]])
k_n_d = 1*np.flipud(k_n_u)

n_neighb = lambda arr, ker: signal.correlate(arr, ker, mode='same')*(arr != 0).astype(int)
n_l = lambda arr: n_neighb(arr, k_n_l)
n_r = lambda arr: n_neighb(arr, k_n_r)
n_u = lambda arr: n_neighb(arr, k_n_u)
n_d = lambda arr: n_neighb(arr, k_n_d)

print_tex('p_o:', puzz,'p : ',puzz_c, 'n_L:',n_l(puzz_c), 'n_R:',n_r(puzz_c), 'n_U:',n_u(puzz_c), 'n_D:',n_d(puzz_c))


<IPython.core.display.Math object>

In [18]:
can_swap = lambda arr, neigbor_f: (neigbor_f(arr)==1).astype(int)
def s_l(arr):
    a = can_swap(arr, n_l)
    a[:,0] *= 0
    return a
def s_r(arr):
    a = can_swap(arr, n_r)
    a[:,-1] *= 0
    return a
def s_u(arr):
    a = can_swap(arr, n_u)
    a[0] *= 0
    return a
def s_d(arr):
    a = can_swap(arr, n_d)
    a[-1] *= 0
    return a


print_tex('p : ',puzz_c, 's_L:',s_l(puzz_c), 's_R:',s_r(puzz_c), 's_U:',s_u(puzz_c), 's_D:',s_d(puzz_c))
mobility = s_l(puzz_c) + s_r(puzz_c) + s_u(puzz_c) + s_d(puzz_c)
print_tex(r'm = \sum_i s_i:',mobility)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Check whish 0s can be swapped

In [7]:
print_tex('p_o:', puzz,'p : ',puzz_c, 's_L:',s_l(1-puzz_c), 's_R:',s_r(1-puzz_c), 's_U:',s_u(1-puzz_c), 's_D:',s_d(1-puzz_c))

<IPython.core.display.Math object>

## Calculate "neighbor energy"
each element gets added +/-1 for having more than 1 neighbor on each axis

* 1s have positive energy
* 0s have negative energy

It done to promote exchange between 1s and 0s if they are close by

In [19]:
n_hor = np.array([  [0,0,0],
                    [1,0,1],
                    [0,0,0]])

n_ver  = np.array([ [0,1,0],
                    [0,0,0],
                    [0,1,0]])
 
nb_cnt_h = lambda arr: signal.convolve2d(arr, n_hor, boundary='fill', mode='same', fillvalue=0)
nb_cnt_v = lambda arr: signal.convolve2d(arr, n_ver, boundary='fill', mode='same', fillvalue=0)

In [25]:
def calc_neighbor_energy0(arr):
    # do calculations only on 'ones' ('zeros' case is 'ones' -1). have mask out 'zeros' in the end
    # calc all vert and horiz neighbors
    # consider only those cases which have more than 1 neighbor (mask others out)
    # sum these vert and horiz cases
    # mask out cases where entries were 0 (they still get 1s as neighbors)
    a1 = arr
    a0 = 1 - arr
    n1h, n1v = nb_cnt_h(a1), nb_cnt_v(a1)
    n0h, n0v = nb_cnt_h(a0), nb_cnt_v(a0)
    e1 = n1h*((n1h > 1).astype(int))/2 + n1v*((n1v > 1).astype(int))/2
    e0 = n0h*((n0h > 1).astype(int))/2 + n0v*((n0v > 1).astype(int))/2
    return  e1*a1 - e0*a0 # make 'zeros' neighbor count negative for gradient

def calc_neighbor_energy(arr):
    a1 = arr
    n1h, n1v = nb_cnt_h(a1), nb_cnt_v(a1)
    e1 = n1h*((n1h > 1).astype(int))/2 + n1v*((n1v > 1).astype(int))/2
    return  e1*a1
e_n = calc_neighbor_energy(puzz_c)
print_tex('p : ',puzz_c, 'e_N:',e_n, 'm:', (mobility>0).astype(int))

<IPython.core.display.Math object>

Extract agitated 1s and their neighbors. Neighbors can be the cause for agitation.

Mask out elements that cannot move.

Add extra weight for violated col/row elem count

In [28]:
cross = np.array([[0, 1, 0],
                  [1, 1, 1],
                  [0, 1, 0]])
a0 = (signal.correlate(e_n, cross, mode='same')>0).astype(int)*(mobility>0).astype(int)
a = a0.copy()
a += (h_constr(puzz_c) > N//2).astype(int) + (v_constr(puzz_c) > N//2).astype(int)
print_tex('p : ',puzz_c,a0, a, a*a0)#*(arr != 0).astype(int)

<IPython.core.display.Math object>

In [30]:
if np.sum(a*a0) == 0:
    candidates = np.argwhere(a == np.max(a))
else:
    candidates = np.argwhere(a*a0 == np.max(a*a0))
candidates

array([[4, 4]], dtype=int64)

Find viable 1s swap options. 

These are 0s neighbors that are mobile

In [32]:
ijplus = np.array([[0,1], [0,-1], [-1,0], [1,0] ])  # y index is reversed

# pad mobility marices to deal with edge cases
s0s = np.zeros(shape = (4,2+N, 2+N))
for i,f in enumerate([s_l, s_r, s_u, s_d]):
    s0s[i,1:-1,1:-1]   = f(1-puzz_c)
print('viable swaps:')
swaps = set()
for i,j in candidates:
    viable_neighbors = [(i + di, j + dj) for (di,dj), s0 in zip(ijplus, s0s) if s0[i + di + 1, j + dj + 1] == 1]
    for n in viable_neighbors:
        swaps.add(((i,j), n))
    print((i,j), viable_neighbors)

#print_tex('p : ',puzz_c)
#print_tex('p : ',puzz_c, s_l(1-puzz_c),s0s[0])   
#print_tex(ijplus)
swaps = list(swaps)
print(swaps)

viable swaps:
(4, 4) [(4, 5)]
[((4, 4), (4, 5))]


In [33]:
for sw in swaps:
    puzz_c2 = puzz_c.copy()
    swap_2_elements(puzz_c2,*sw)
    h_constr = np.sum(puzz_c2, axis= 1, keepdims=True)-3
    v_constr = np.sum(puzz_c2, axis= 0, keepdims=True)-3
    rc_constr = np.sum(np.abs(h_constr)+np.abs(v_constr))/6
    print_tex(str(sw),r'\ p:', puzz_c,'p_2 : ',puzz_c2, calc_neighbor_energy(puzz_c2), np.sum(calc_neighbor_energy(puzz_c2)), '-',h_constr,v_constr, '-',rc_constr)

<IPython.core.display.Math object>

In [115]:
k_div_x_l = np.array([[0 , 0, -0],
                      [1, -1,  0],
                      [0 , 0, -0]])
k_div_x_r = 1*np.fliplr(k_div_x_l)

k_div_y_u = np.array([[0 , 1, -0],
                      [0, -1,  0],
                      [0 , 0, -0]])
k_div_y_d = 1*np.flipud(k_div_y_u)


div_x_l = lambda arr, mask = None: signal.correlate(arr, k_div_x_l, mode='same')*((mask if mask is not None else arr) != 0).astype(int)
div_x_r = lambda arr, mask = None: signal.correlate(arr, k_div_x_r, mode='same')*((mask if mask is not None else arr) != 0).astype(int)
div_y_u = lambda arr, mask = None: signal.correlate(arr, k_div_y_u, mode='same')*((mask if mask is not None else arr) != 0).astype(int)
div_y_d = lambda arr, mask = None: signal.correlate(arr, k_div_y_d, mode='same')*((mask if mask is not None else arr) != 0).astype(int)
#div_y = lambda arr: signal.correlate(arr, k_div_y, mode='same')
print_tex(r'x_l = ',k_div_x_l, r'x_r = ', k_div_x_r, r'y_u = ', k_div_y_u, r'y_d = ', k_div_y_d)
print_tex(r'p :', e_n, r'x_l :', div_x_l(e_n), r'x_r :', div_x_r(e_n))
print_tex(r'p :', e_n, r'y_u :', div_y_u(e_n), r'y_d :',div_y_d(e_n))
#print_tex(-div_y(puzz_n))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Number of "elements energy" constraint 
Rows with large number of elements should have high energy and opposite with small numbers

In [116]:
cnt_v1 = (np.sum(puzz_c, axis= 0, keepdims=True)>N//2).astype(int)
cnt_v0 = (np.sum(puzz_c, axis= 0, keepdims=True)<N//2).astype(int)
cnt_h1 = (np.sum(puzz_c, axis= 1, keepdims=True)>N//2).astype(int)
cnt_h0 = (np.sum(puzz_c, axis= 1, keepdims=True)<N//2).astype(int)
puzz_n_cnt = e_n + cnt_v1 + cnt_h1 - cnt_v0 - cnt_h0
print_tex(puzz_n_cnt)

<IPython.core.display.Math object>

In [117]:
print_tex(r'x_l = ',k_div_x_l, r'x_r = ', k_div_x_r, r'y_u = ', k_div_y_u, r'y_d = ', k_div_y_d)
print_tex(r'p :', puzz_n_cnt, r'x_l :', div_x_l(puzz_n_cnt,e_n), r'x_r :', div_x_r(puzz_n_cnt,e_n))
print_tex(r'p :', puzz_n_cnt, r'y_u :', div_y_u(puzz_n_cnt,e_n), r'y_d :', div_y_d(puzz_n_cnt,e_n))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>